Copyright 2019 The TensorFlow Authors.

# Neural machine translation with attention

구글에서 제공하는 seq2seq + attention 모델 튜토리얼 입니다.

99 퍼센트의 코드는 그대로 가져왔습니다 

이 모델을 사용하여, Spanish 를 English 로 번역하는 모델을 생성하게 됩니다.

## 사전 준비

### 필요 패키지 import

In [25]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [26]:
# GPU load 문제가 생길때
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

### Dataset Download

http://www.manythings.org/anki/ 에서 제공하는 데이터셋을 사용합니다

tf.keras.utils 의 기본 dataset directory 에 저장됩니다

In [2]:
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

### Cleansing Dataset

 w = w.rstrip().strip()
- 공백 제거시에 왜 오른쪽 공백 제거후 양쪽 공백 제거를 하는지 모르겠음.
- 그냥 모두 처리후에 공백 2칸 이상인 부분 제거를 하면 될듯.

In [3]:
# Converts the unicode file to ascii
# Unicode 파일을 ascii 로 변환

def unicode_to_ascii(s):
      return ''.join(c for c in unicodedata.normalize('NFD', s)
          if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # 단어와 마침표 사이에 white space 를 추가합니다
    # eg: "he is a boy." => "he is a boy ."
    
    # 해당 특수문자 앞뒤에 space 추가합니다.
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    
    # 빈칸이 2칸 이상인 경우를 전부 한칸으로 바꿉니다.
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    # 알파벳과 주로 쓰이는 특수문자 외의 것들은 모두 빈칸으로 처리합니다
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    #  오른쪽 공백 제거 후 양쪽 공백 제거
    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    
    # 모든 문장에 시작과 끝을 알리는 토큰을 추가합니다
    # 시작과 끝을 알려주는 토큰이 있어야 모델이 학습시 문장의 시작과 끝을 판단할 수 있습니다.
    w = '<start> ' + w + ' <end>'
    return w

#### Data cleansing test

In [4]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
<start> ¿ puedo tomar prestado este libro ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


### Build Dataset


Google API 에서 받아온 데이터를 모델에 넣기 용이한 데이터셋으로 변환하는 작업.

데이터셋의 구조에 따라 달라서 넘어가도 될듯

In [5]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]

def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    # zip(*) 를 사용한것은,  word_pairs 를 먼저 unpacking 한 뒤에 zip 을 해야하기 때문에
    return zip(*word_pairs)

In [6]:
en, sp = create_dataset(path_to_file, None)

#### Data Check

In [7]:
print(en[-1])
print(sp[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


#### Dataset 생성용 함수들

In [8]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [9]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

In [10]:
def load_dataset(path, num_examples=None):

    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

#### Data 길이 제한 (Optional)

학습 속도를 빠르게 하기 위해 데이터 길이를 제한할 수 있음.

하지만 모델의 성능이 떨어질 수 있음.

In [11]:
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [12]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

24000 24000 6000 6000


In [13]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [14]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
4979 ----> tranquilizate
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
600 ----> calm
128 ----> down
3 ----> .
2 ----> <end>


####  tf.data dataset 생성

Data Input shape 을 확인하기 위해 example_input_batch 생성해서 확인

In [15]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [16]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 11]))

## Encoder / Decoder 모델

### 기본 설명

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

왼쪽의 Encoder 에 들어간 Input:

- `(batch_size, max_length, hidden_size)` 의 `encoder output`

- `(batch_size, hidden_size)`             의 `encoder hidden state`

를 리턴합니다.

이 값들로 Attention weight 를 계산하여 Context Vector 를 계산합니다.

복잡한 수식은 패스하고, 직관적으로 보겠습니다.

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))` Encoder Output 과 hidden state로 attention score 를 계산합니다


* `attention weights = softmax(score, axis = 1)`. 기본적으로 Softmax 는 마지막 axis (-1) 에 적용되지만, 이 예시에서는 첫번째 axis 에 적용합니다. score의 shape 이 *(batch_size, max_length, hidden_size)* 이기 때문입니다. `Max_length` 는 우리가 넣은 input의 길이 입니다. 모든 입력값에 weight 을 적용시키려 하기 때문에, softmax 가 axis 1에 적용됩니다.


* `context vector = sum(attention weights * EO, axis = 1)`. 여기도 axis 1에 적용합니다


* `embedding output` = Decoder 에 들어가는 입력도 embedding layer 를 통해 들어갑니다.


* `merged vector = concat(embedding output, context vector)`


* 이 merged vector 가 GRU 에 주어집니다.

### Encoder Model

Embedding layer -> GRU 

In [17]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [19]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input 이용해서 shape 확인
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 16, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


### Attention layer

`score = FC(tanh(FC(EO) + FC(H)))`

In [23]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        
        # tanh(FC(EO)) 를 위한 dense layer
        self.W1 = tf.keras.layers.Dense(units)
        
        # FC(H) 를 위한 dense layer
        self.W2 = tf.keras.layers.Dense(units)
        
        # Score 를 최종 계산하기 위한 dense layer
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        
        # query 는 encoder hidden state
        # values 는 encoder output
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # 마지막에 units = 1인 FC 를 사용하여 score 의 axis=2 가 1이됨
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [27]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 16, 1)
